In [1]:
%matplotlib inline
import os
import random
import pickle
#import cPickle as pickle
import numpy as np
import matplotlib.pyplot
from matplotlib.pyplot import imshow
import keras
from keras.preprocessing import image
# from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from sklearn.decomposition import PCA
from scipy.spatial import distance
from tqdm import tqdm


/Users/EricX/anaconda2/envs/341/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import sys
sys.executable

'/Users/EricX/anaconda2/envs/341/bin/python'

In [3]:
# model = keras.applications.VGG16(weights='imagenet', include_top=True)
# model.summary()
import util.util as data_util


In [4]:
batches = data_util.batch_generator(img_size=[224, 224])

(True, True)
Loading data completed


In [5]:
X_src_batch, X_tar_batch, score_batch = batches[0]
IMG_SHAPE = X_src_batch.shape[1:]
print(X_src_batch.shape, X_tar_batch.shape, score_batch.shape)

((50, 224, 224, 3), (50, 224, 224, 3), (50,))


In [6]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout, concatenate
model = keras.applications.VGG16(weights='imagenet', include_top=True)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
feature_model = Model(inputs=model.input, outputs=model.get_layer("block5_pool").output, name = 'FeatureGenerationModel')
feature_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [14]:
src_in = Input(shape = IMG_SHAPE, name = 'src_input')
tar_in = Input(shape = IMG_SHAPE, name = 'tar_input')
src_feat = feature_model(src_in)
tar_feat = feature_model(tar_in)

In [17]:
tar_feat.shape
test = Dense(2)(tar_feat)
test.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(2)])

In [9]:
combined_features = concatenate([src_feat, tar_feat], name = 'merge_features')
combined_features = Dense(1024, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1024, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
#A trick for bounded output range is to scale the target values between (0,1) and use sigmoid output + binary cross-entropy loss.
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = Model(inputs = [src_in, tar_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
src_input (InputLayer)          (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
tar_input (InputLayer)          (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
FeatureGenerationModel (Model)  (None, 4096)         134260544   src_input[0][0]                  
                                                                 tar_input[0][0]                  
__________________________________________________________________________________________________
merge_features (Concatenate)    (None, 8192)         0           FeatureGenerationModel[1][0]     
          

In [10]:
# setup the optimization process
#A trick for bounded output range is to scale the target values between (0,1) and use sigmoid output + binary cross-entropy loss.
similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['mae'])



In [11]:
# def show_model_output(nb_examples = 3):
#     pv_a, pv_b, pv_sim = gen_random_batch(test_groups, nb_examples)
#     pred_sim = similarity_model.predict([pv_a, pv_b])
#     fig, m_axs = plt.subplots(2, pv_a.shape[0], figsize = (12, 6))
#     for c_a, c_b, c_d, p_d, (ax1, ax2) in zip(pv_a, pv_b, pv_sim, pred_sim, m_axs.T):
#         ax1.imshow(c_a[:,:,0])
#         ax1.set_title('Image A\n Actual: %3.0f%%' % (100*c_d))
#         ax1.axis('off')
#         ax2.imshow(c_b[:,:,0])
#         ax2.set_title('Image B\n Predicted: %3.0f%%' % (100*p_d))
#         ax2.axis('off')
#     return fig
# # a completely untrained model
# _ = show_model_output()

In [12]:
# make a generator out of the data
def data_generator(batches, batch_size=100):
    #batches = data_util.batch_generator(batch_size=batch_size)
    while True:
        for batch in batches:
            yield ([batch[0], batch[1]], batch[2])
# we want a constant validation group to have a frame of reference for model performance
valid_a, valid_b, valid_sim = batches[0]
loss_history = similarity_model.fit_generator(data_generator(batches),
                               validation_data=([valid_a, valid_b], valid_sim),
                                steps_per_epoch=len(batches),
                                epochs = 10,
                                verbose = True)

Epoch 1/10
14/20 [====================>.........] - ETA: 8:41 - loss: 0.3170 - mean_absolute_error: 0.1609 

KeyboardInterrupt: 